In [1]:
import numpy as np

from qonnx.core.datatype import DataType
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes

from qonnx.util.cleanup import cleanup as qonnx_cleanup

from finn.util.visualization import showInNetron
from qonnx.core.modelwrapper import ModelWrapper

from qonnx.custom_op.registry import getCustomOp

import onnx.helper as oh
import qonnx.util.basic as util

In [2]:
prune_folder = './manual_prune/'

# Load Model and Clean

In [3]:
model_file = './MY_MBLNET_V2_RESNET_classifier__best_mean_F1__BIPOLAR_Out__QONNX.onnx'

In [4]:
qonnx_clean_filename = prune_folder + '00_prune_clean.onnx'
qonnx_cleanup(model_file, out_file=qonnx_clean_filename)

In [5]:
showInNetron(qonnx_clean_filename)

Serving './manual_prune/00_prune_clean.onnx' at http://0.0.0.0:8083


# Analyze layers to prune

In [6]:
model = ModelWrapper(qonnx_clean_filename)

In [7]:
all_inits_names = [init.name for init in model.graph.initializer]

print(f'Number of initializers = {len(all_inits_names)}')

Number of initializers = 376


In [8]:
eps = 1e-10

layers_to_prune = {}

for idx, init_name in enumerate(all_inits_names):
    if "Quant" in init_name and "param1" in init_name:
    # It is a scale value, check it
        np_init = model.get_initializer(init_name)
        np_abs_val = np.abs(np_init)
        zero_idx = (np_abs_val < eps) * (np_abs_val > 0)
        if np.all(zero_idx == False):
            #print(f'Index = {idx}. {init_name} was not appended, as there were no values under epsilon')
            continue
        else:
            zero_layer = np.where(zero_idx == True)[0]
            quant_layer_out_name = init_name.split("param")[0] + "out0"
            layers_to_prune[quant_layer_out_name] = {1: {*zero_layer}}
            #print(f'Index = {idx}. {init_name} appended, as there were values under epsilon')

In [9]:
print(f'Number of layers to prune: {len(layers_to_prune)}')
for k, v in layers_to_prune.items():
    print(k, v)

Number of layers to prune: 19
Quant_0_out0 {1: {17}}
Quant_1_out0 {1: {17}}
Quant_6_out0 {1: {31}}
Quant_7_out0 {1: {31}}
Quant_12_out0 {1: {36, 6, 40, 9, 43, 12, 46, 17, 26, 28}}
Quant_13_out0 {1: {36, 6, 40, 9, 43, 12, 46, 17, 26, 28}}
Quant_15_out0 {1: {0, 65, 2, 3, 67, 75, 14, 81, 18, 50, 52, 93, 22, 23, 88, 92, 61}}
Quant_16_out0 {1: {0, 65, 2, 3, 67, 75, 14, 81, 18, 50, 52, 93, 22, 23, 88, 92, 61}}
Quant_18_out0 {1: {3, 6, 8, 9, 11, 16, 17, 18, 25, 26, 33, 37, 39, 42, 52, 54, 62, 63, 71, 72, 74, 82, 84, 86, 90, 92, 93, 94, 99, 100, 102, 107, 111, 113, 116, 123, 124, 125, 126}}
Quant_19_out0 {1: {3, 6, 8, 9, 11, 16, 17, 18, 25, 26, 33, 37, 39, 42, 52, 54, 62, 63, 71, 72, 74, 82, 84, 86, 90, 92, 93, 94, 99, 100, 102, 107, 111, 113, 116, 123, 124, 125, 126}}
Quant_21_out0 {1: {0, 4, 5, 7, 13, 16, 17, 20, 21, 27, 30, 31, 33, 34, 35, 37, 40, 41, 49, 53, 60, 61, 62, 63, 64, 68, 71, 72, 76, 78, 79, 80, 82, 85, 88, 90, 91, 93, 95, 100, 102, 103, 105, 106, 107, 108, 116, 117, 120, 121, 12

# Prune the Layers Manually

#### Conv1

In [10]:
def prune_conv_non_dw_0(model, conv_id):

    str_conv_id = str(conv_id)

    print("Convert Quant Weights of Convolution")
    quant_node = "Quant_" + str_conv_id
    quant_0 = quant_node + "_param0"
    quant_1 = quant_node + "_param1"
    np_q0 = model.get_initializer(quant_0)
    np_q1 = model.get_initializer(quant_1)
    print(f'Quant 0 shape: {np_q0.shape}')
    print(f'Quant 1 shape: {np_q1.shape}') 

    np_q1_abs = np.abs(np_q1)
    non_zero_idx = np.where(np_q1_abs > 1e-10)[0]
    print(f'Non Zero IDX: {non_zero_idx}')

    new_np_q0 = np_q0[non_zero_idx]
    print(f'New Quant 0 shape: {new_np_q0.shape}')
    new_np_q1 = np_q1[non_zero_idx]
    print(f'New Quant 1 shape: {new_np_q1.shape}')

    model.set_initializer(
        tensor_name = quant_0, 
        tensor_value = new_np_q0)
    model.set_initializer(
        tensor_name = quant_1, 
        tensor_value = new_np_q1)

    node_q0 = model.get_node_from_name(quant_node)
    ch, k, w, h = model.get_tensor_shape(node_q0.output[0])
    print(f'Quant 0 original shape: {ch, k, w, h}')
    new_ch = new_np_q0.shape[0]
    new_shape = (new_ch, k, w, h)
    print(f'Quant 0 new shape: {new_shape}')

    model.set_tensor_shape(node_q0.output[0], new_shape)

    print("Convert Convolution Shape")

    conv_node = "Conv_" + str_conv_id
    node_conv0 = model.get_node_from_name(conv_node)
    batch, ch, w, h = model.get_tensor_shape(node_conv0.output[0])
    print(batch, ch, w, h)
    new_ch = new_np_q0.shape[0]
    new_shape = (batch, new_ch, w, h)
    print(new_shape)

    model.set_tensor_shape(node_conv0.output[0], new_shape)

    print("Convert Batch Norm")
    bn_node = "BatchNormalization_" + str_conv_id
    bn_0 = "BatchNormalization_" + str_conv_id + "_param0"
    bn_1 = "BatchNormalization_" + str_conv_id + "_param1"
    bn_2 = "BatchNormalization_" + str_conv_id + "_param2"
    bn_3 = "BatchNormalization_" + str_conv_id + "_param3"
    np_bn0 = model.get_initializer(bn_0)
    np_bn1 = model.get_initializer(bn_1)
    np_bn2 = model.get_initializer(bn_2)
    np_bn3 = model.get_initializer(bn_3)
    
    print(f'BN0 shape: {np_bn0.shape}')
    print(f'BN1 shape: {np_bn1.shape}')
    print(f'BN2 shape: {np_bn2.shape}')
    print(f'BN3 shape: {np_bn3.shape}')

    np_bn0_abs = np.abs(np_bn0)
    non_zero_idx = np.where(np_bn0_abs > 1e-10)[0]
    print(f'BN non zero IDX: {non_zero_idx}')
    new_np_bn0 = np_bn0[non_zero_idx]
    print(new_np_bn0.shape)
    new_np_bn1 = np_bn1[non_zero_idx]
    print(new_np_bn1.shape)
    new_np_bn2 = np_bn2[non_zero_idx]
    print(new_np_bn2.shape)
    new_np_bn3 = np_bn3[non_zero_idx]
    print(new_np_bn3.shape)

    model.set_initializer(
        tensor_name = bn_0, 
        tensor_value = new_np_bn0)
    model.set_initializer(
        tensor_name = bn_1, 
        tensor_value = new_np_bn1)
    model.set_initializer(
        tensor_name = bn_2, 
        tensor_value = new_np_bn2)
    model.set_initializer(
        tensor_name = bn_3, 
        tensor_value = new_np_bn3) 

    node_bn0 = model.get_node_from_name(bn_node)
    model.set_tensor_shape(node_bn0.output[0], new_shape) 

    print("Convert ReLU")
    relu_node = "Relu_" + str_conv_id
    node_relu0 = model.get_node_from_name(relu_node)  
    model.set_tensor_shape(node_relu0.output[0], new_shape)

    print("Convert Quant ReLU")
    quant_relu_node = "Quant_34"
    quant_node_relu0 = model.get_node_from_name(quant_relu_node)  
    model.set_tensor_shape(quant_node_relu0.output[0], new_shape)    

In [11]:
prune_conv_non_dw_0(model, 0)

Convert Quant Weights of Convolution
Quant 0 shape: (32, 3, 3, 3)
Quant 1 shape: (32, 1, 1, 1)
Non Zero IDX: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
New Quant 0 shape: (31, 3, 3, 3)
New Quant 1 shape: (31, 1, 1, 1)
Quant 0 original shape: (32, 3, 3, 3)
Quant 0 new shape: (31, 3, 3, 3)
Convert Convolution Shape
1 32 112 112
(1, 31, 112, 112)
Convert Batch Norm
BN0 shape: (32,)
BN1 shape: (32,)
BN2 shape: (32,)
BN3 shape: (32,)
BN non zero IDX: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
(31,)
(31,)
(31,)
(31,)
Convert ReLU
Convert Quant ReLU


In [12]:
model = model.transform(InferShapes())

In [13]:
prune_first_conv = prune_folder + "01_prune_first_conv.onnx"
model.save(prune_first_conv)

In [14]:
showInNetron(prune_first_conv)

Stopping http://0.0.0.0:8083
Serving './manual_prune/01_prune_first_conv.onnx' at http://0.0.0.0:8083


In [15]:
def prune_conv_dw_1(model, conv_id):

    str_conv_id = str(conv_id)

    print("Convert Quant Weights of Convolution")
    quant_node = "Quant_" + str_conv_id
    quant_0 = quant_node + "_param0"
    quant_1 = quant_node + "_param1"
    np_q0 = model.get_initializer(quant_0)
    np_q1 = model.get_initializer(quant_1)
    print(f'Quant 0 shape: {np_q0.shape}')
    print(f'Quant 1 shape: {np_q1.shape}') 

    np_q1_abs = np.abs(np_q1)
    non_zero_idx = np.where(np_q1_abs > 1e-10)[0]
    print(f'Non Zero IDX: {non_zero_idx}')

    new_np_q0 = np_q0[non_zero_idx]
    print(f'New Quant 0 shape: {new_np_q0.shape}')
    new_np_q1 = np_q1[non_zero_idx]
    print(f'New Quant 1 shape: {new_np_q1.shape}')

    model.set_initializer(
        tensor_name = quant_0, 
        tensor_value = new_np_q0)
    model.set_initializer(
        tensor_name = quant_1, 
        tensor_value = new_np_q1)

    node_q0 = model.get_node_from_name(quant_node)
    ch, k, w, h = model.get_tensor_shape(node_q0.output[0])
    print(f'Quant 0 original shape: {ch, k, w, h}')
    new_ch = new_np_q0.shape[0]
    new_shape = (new_ch, k, w, h)
    print(f'Quant 0 new shape: {new_shape}')

    model.set_tensor_shape(node_q0.output[0], new_shape)

    print("Convert Convolution Shape")

    conv_node = "Conv_" + str_conv_id
    node_conv0 = model.get_node_from_name(conv_node)
    batch, ch, w, h = model.get_tensor_shape(node_conv0.output[0])
    print(batch, ch, w, h)
    new_ch = new_np_q0.shape[0]
    new_shape = (batch, new_ch, w, h)
    print(new_shape)

    model.set_tensor_shape(node_conv0.output[0], new_shape)

    # Change groups
    conv_group = node_conv0.attribute[1].i
    if conv_group != 1:
        print(f'DW Conv found. Group {conv_group}, changed to {new_ch}')
        node_conv0.attribute[1].i = new_ch
    
    print("Convert Batch Norm")
    bn_node = "BatchNormalization_" + str_conv_id
    bn_0 = "BatchNormalization_" + str_conv_id + "_param0"
    bn_1 = "BatchNormalization_" + str_conv_id + "_param1"
    bn_2 = "BatchNormalization_" + str_conv_id + "_param2"
    bn_3 = "BatchNormalization_" + str_conv_id + "_param3"
    np_bn0 = model.get_initializer(bn_0)
    np_bn1 = model.get_initializer(bn_1)
    np_bn2 = model.get_initializer(bn_2)
    np_bn3 = model.get_initializer(bn_3)
    
    print(f'BN0 shape: {np_bn0.shape}')
    print(f'BN1 shape: {np_bn1.shape}')
    print(f'BN2 shape: {np_bn2.shape}')
    print(f'BN3 shape: {np_bn3.shape}')

    np_bn0_abs = np.abs(np_bn0)
    non_zero_idx = np.where(np_bn0_abs > 1e-10)[0]
    print(f'BN non zero IDX: {non_zero_idx}')
    new_np_bn0 = np_bn0[non_zero_idx]
    print(new_np_bn0.shape)
    new_np_bn1 = np_bn1[non_zero_idx]
    print(new_np_bn1.shape)
    new_np_bn2 = np_bn2[non_zero_idx]
    print(new_np_bn2.shape)
    new_np_bn3 = np_bn3[non_zero_idx]
    print(new_np_bn3.shape)

    model.set_initializer(
        tensor_name = bn_0, 
        tensor_value = new_np_bn0)
    model.set_initializer(
        tensor_name = bn_1, 
        tensor_value = new_np_bn1)
    model.set_initializer(
        tensor_name = bn_2, 
        tensor_value = new_np_bn2)
    model.set_initializer(
        tensor_name = bn_3, 
        tensor_value = new_np_bn3) 

    node_bn0 = model.get_node_from_name(bn_node)
    model.set_tensor_shape(node_bn0.output[0], new_shape) 

    print("Convert ReLU")
    relu_node = "Relu_" + str_conv_id
    node_relu0 = model.get_node_from_name(relu_node)  
    model.set_tensor_shape(node_relu0.output[0], new_shape)

    print("Convert Quant ReLU")
    quant_relu_node = "Quant_35"
    quant_node_relu0 = model.get_node_from_name(quant_relu_node)  
    model.set_tensor_shape(quant_node_relu0.output[0], new_shape)  

In [16]:
# conv1_node = model.get_node_from_name("Conv_1")  

In [17]:
# print(conv1_node)

In [18]:
# for attr in conv1_node.attribute:
#     print(type(attr))
#     new_attr = oh.get_attribute_value(attr)
#     print(new_attr)

In [19]:
# conv_1_group = util.get_by_name(conv1_node.attribute, "group")

In [20]:
# conv_1_group

In [21]:
# conv1_node.attribute[1].i = 31

In [22]:
prune_conv_dw_1(model, 1)

Convert Quant Weights of Convolution
Quant 0 shape: (32, 1, 3, 3)
Quant 1 shape: (32, 1, 1, 1)
Non Zero IDX: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
New Quant 0 shape: (31, 1, 3, 3)
New Quant 1 shape: (31, 1, 1, 1)
Quant 0 original shape: (32, 1, 3, 3)
Quant 0 new shape: (31, 1, 3, 3)
Convert Convolution Shape
1 32 112 112
(1, 31, 112, 112)
DW Conv found. Group 32, changed to 31
Convert Batch Norm
BN0 shape: (32,)
BN1 shape: (32,)
BN2 shape: (32,)
BN3 shape: (32,)
BN non zero IDX: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
(31,)
(31,)
(31,)
(31,)
Convert ReLU
Convert Quant ReLU


In [23]:
model = model.transform(InferShapes())

In [24]:
prune_first_conv = prune_folder + "02_prune_first_conv.onnx"
model.save(prune_first_conv)

In [25]:
showInNetron(prune_first_conv)

Stopping http://0.0.0.0:8083
Serving './manual_prune/02_prune_first_conv.onnx' at http://0.0.0.0:8083


In [10]:
np_init_0 = model.get_initializer("Quant_0_param0")
np_init_1 = model.get_initializer("Quant_0_param1")

print(np_init_0.shape)
print(np_init_1.shape)

(32, 3, 3, 3)
(32, 1, 1, 1)


In [11]:
np_init_1_abs = np.abs(np_init_1)
zero_idx = np.where((np_init_1_abs < 1e-10) * (np_init_1_abs > 0))[0]
print(zero_idx)
# print(1-zero_idx)

[17]


In [12]:
print(zero_idx.shape)

(1,)


In [13]:
print(zero_idx.shape)
new_np_init_0 = np_init_0[zero_idx]
print(new_np_init_0.shape)

(1,)
(1, 3, 3, 3)


In [14]:
non_zero_idx = np.where(np_init_1_abs > 1e-10)[0]
print(non_zero_idx)
# print(1-zero_idx)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]


In [15]:
new_np_init_0 = np_init_0[non_zero_idx]
print(new_np_init_0.shape)
new_np_init_1 = np_init_1[non_zero_idx]
print(new_np_init_1.shape)

(31, 3, 3, 3)
(31, 1, 1, 1)


In [16]:
model.set_initializer(
    tensor_name = "Quant_0_param0", 
    tensor_value = new_np_init_0)

In [17]:
model.set_initializer(
    tensor_name = "Quant_0_param1", 
    tensor_value = new_np_init_1)

In [18]:
node_q0 = model.get_node_from_name("Quant_0")
ch, k, w, h = model.get_tensor_shape(node_q0.output[0])
print(ch, k, w, h)
new_ch = new_np_init_0.shape[0]
new_shape = (new_ch, k, w, h)
print(new_shape)

32 3 3 3
(31, 3, 3, 3)


In [19]:
model.set_tensor_shape(node_q0.output[0], new_shape)

In [20]:
node_conv0 = model.get_node_from_name("Conv_0")
batch, ch, w, h = model.get_tensor_shape(node_conv0.output[0])
print(batch, ch, w, h)
new_ch = new_np_init_0.shape[0]
new_shape = (batch, new_ch, w, h)
print(new_shape)

1 32 112 112
(1, 31, 112, 112)


In [21]:
model.set_tensor_shape(node_conv0.output[0], new_shape)

In [22]:
prune_first_conv = prune_folder + "manual_prune_first_conv.onnx"
model.save(prune_first_conv)

In [23]:
showInNetron(prune_first_conv)

Stopping http://0.0.0.0:8083
Serving './manual_prune/manual_prune_first_conv.onnx' at http://0.0.0.0:8083


In [26]:
np_init_bn0 = model.get_initializer("BatchNormalization_0_param0")
np_init_bn1 = model.get_initializer("BatchNormalization_0_param1")
np_init_bn2 = model.get_initializer("BatchNormalization_0_param2")
np_init_bn3 = model.get_initializer("BatchNormalization_0_param3")

print(np_init_bn0.shape)
print(np_init_bn1.shape)
print(np_init_bn2.shape)
print(np_init_bn3.shape)

(32,)
(32,)
(32,)
(32,)


In [27]:
np_init_bn0_abs = np.abs(np_init_bn0)
zero_idx = np.where((np_init_bn0_abs < 1e-10) * (np_init_bn0_abs > 0))[0]
print(zero_idx)
# print(1-zero_idx)

[17]


In [28]:
non_zero_idx = np.where(np_init_bn0_abs > 1e-10)[0]
print(non_zero_idx)
# print(1-zero_idx)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]


In [29]:
new_np_init_bn0 = np_init_bn0[non_zero_idx]
print(new_np_init_bn0.shape)
new_np_init_bn1 = np_init_bn1[non_zero_idx]
print(new_np_init_bn1.shape)
new_np_init_bn2 = np_init_bn2[non_zero_idx]
print(new_np_init_bn2.shape)
new_np_init_bn3 = np_init_bn3[non_zero_idx]
print(new_np_init_bn3.shape)

(31,)
(31,)
(31,)
(31,)


In [30]:
model.set_initializer(
    tensor_name = "BatchNormalization_0_param0", 
    tensor_value = new_np_init_bn0)
model.set_initializer(
    tensor_name = "BatchNormalization_0_param1", 
    tensor_value = new_np_init_bn1)
model.set_initializer(
    tensor_name = "BatchNormalization_0_param2", 
    tensor_value = new_np_init_bn2)
model.set_initializer(
    tensor_name = "BatchNormalization_0_param3", 
    tensor_value = new_np_init_bn3)

In [32]:
node_bn0 = model.get_node_from_name("BatchNormalization_0")
batch, ch, w, h = model.get_tensor_shape(node_bn0.output[0])
print(batch, ch, w, h)
new_ch = new_np_init_bn0.shape[0]
new_shape = (batch, new_ch, w, h)
print(new_shape)

1 32 112 112
(1, 31, 112, 112)


In [33]:
model.set_tensor_shape(node_bn0.output[0], new_shape)

In [34]:
prune_first_conv = prune_folder + "01_manual_prune_first_conv.onnx"
model.save(prune_first_conv)

In [35]:
showInNetron(prune_first_conv)

Stopping http://0.0.0.0:8083
Serving './manual_prune/01_manual_prune_first_conv.onnx' at http://0.0.0.0:8083


In [36]:
node_relu0 = model.get_node_from_name("Relu_0")
batch, ch, w, h = model.get_tensor_shape(node_relu0.output[0])
print(batch, ch, w, h)
new_ch = new_np_init_bn0.shape[0]
new_shape = (batch, new_ch, w, h)
print(new_shape)

1 32 112 112
(1, 31, 112, 112)


In [37]:
model.set_tensor_shape(node_relu0.output[0], new_shape)

In [41]:
node_q34 = model.get_node_from_name("Quant_34")
batch, ch, w, h = model.get_tensor_shape(node_q34.output[0])
print(batch, ch, w, h)
new_ch = new_np_init_bn0.shape[0]
new_shape = (batch, new_ch, w, h)
print(new_shape)

1 32 112 112
(1, 31, 112, 112)


In [42]:
model.set_tensor_shape(node_q34.output[0], new_shape)

In [43]:
prune_first_conv = prune_folder + "02_manual_prune_first_conv.onnx"
model.save(prune_first_conv)

In [44]:
showInNetron(prune_first_conv)

Stopping http://0.0.0.0:8083
Serving './manual_prune/02_manual_prune_first_conv.onnx' at http://0.0.0.0:8083


In [45]:
model = model.transform(InferShapes())

In [46]:
prune_first_conv = prune_folder + "03_manual_prune_first_conv.onnx"
model.save(prune_first_conv)

In [17]:
showInNetron(prune_first_conv)

Stopping http://0.0.0.0:8083
Serving './manual_prune/01_prune_first_conv.onnx' at http://0.0.0.0:8083


In [27]:
model = model.transform(InferShapes())

InferenceError: [ShapeInferenceError] (op_type:Conv, node name: Conv_0): [ShapeInferenceError] Inferred shape and existing shape differ in dimension 1: (31) vs (32)

In [76]:
model.save("manual_prune_test.onnx")

In [77]:
showInNetron("manual_prune_test.onnx")

Stopping http://0.0.0.0:8083
Serving 'manual_prune_test.onnx' at http://0.0.0.0:8083
